In [2]:
import urllib.request
from bs4 import BeautifulSoup, NavigableString
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
BASE_PEOPLE = "https://letterboxd.com/people/popular/page/"
LBOX = "https://letterboxd.com"
REV = "films/reviews/page/"

userNames = list();

t0 = time.time()

for i in range (1,2):
    url = BASE_PEOPLE + str(i);
    soup = BeautifulSoup(urllib.request.urlopen(url),"lxml")
    for tag in soup.find_all("h3", "title-3"):
        href = tag.a.get("href")
        userNames.append(href)
    
    time.sleep(0.25)
t1 = time.time()            
print(f"{t1-t0} seconds to download {len(userNames)} usernames.")

2.3921098709106445 seconds to download 35 usernames.


In [4]:
len(userNames)

35

In [5]:
# UNCOMMENT to install: !pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())  #using Selenium-Chrome simulator to interact with JS


Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\tanmu\AppData\Roaming\SPB_16.6\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]


In [6]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
# to store num of pages of reviews to usernames
numPageReviews = dict()
# to store all data of reviews to usernames
allReviewData = dict()

driver = webdriver.Chrome(ChromeDriverManager().install()) 
t0 = time.time()

for name in userNames[:1]:
    #Find num of pages of reviews from the first page
    firstPage = LBOX + name + REV + "1";
    
    soup = BeautifulSoup(urllib.request.urlopen(firstPage),"lxml")
    time.sleep(2)
    
    # to store user's all review data (movie/film name + year + review + rating)
    data = list()

    for tag in soup.find_all("div", "paginate-pages"):
        item = tag.find_all('li','paginate-page')
        numPageReviews[name] = (int(item[-1].text))
    
    # initialize lists for data
    ratings = list()
    movies = list()
    reviews = list()
    
    # Loop through all pages and obtain data from each page
    for pageNum in range(1, numPageReviews[name]+1):
        thePage = LBOX + name + REV + str(pageNum+7);
        driver.get(thePage)
        time.sleep(5)
        buttons = None
        buttons = driver.find_elements_by_class_name("reveal")
        time.sleep(1)
        state = True
        if len(buttons)==0:
            state = False
        while(state): # Click on "more" to reveal all text
            try :
                button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, "reveal")))
                button.click()
            except TimeoutException:
                pass
            if len(driver.find_elements_by_class_name("reveal"))==0:
                state = False     
        soup = BeautifulSoup(driver.page_source,"html.parser")
        time.sleep(1)
        ratingList = []  # list to store the review entries with ratings only      
        for tag in soup.find_all("p", "attribution"):
            spans = tag.find_all('span','rating')
            ratingList.append(spans)
            for span in spans:
                rating = [i for i in str(span.attrs.get('class')) if i in "0123456789"]
                if len(rating)==2:
                    rating = ['10']
                ratings.append(rating[0])

        for i, tag in enumerate(soup.find_all("h2", "headline-2 prettify")):
            if not ratingList[i]:
                continue
            filmName = tag.a
            filmYear = tag.small.a
            for film in zip(filmName, filmYear):
                movies.append(film)
        

        for i, tag in enumerate(soup.find_all("div", "body-text")):
            if not ratingList[i]:
                continue
            rev = "";
            for item in tag:
                if isinstance(item, NavigableString):
                    continue
                rev += str(item.text)
            reviews.append(cleanhtml(rev))
            
    for i in range(len(ratings)):
        data.append([movies[i],reviews[i],ratings[i]])
        
    allReviewData[name] = data;
    
t1 = time.time()


Looking for [chromedriver 79.0.3945.36 win32] driver in cache 
File found in cache by path [C:\Users\tanmu\AppData\Roaming\SPB_16.6\.wdm\drivers\chromedriver\79.0.3945.36\win32\chromedriver.exe]
